In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/pbp_related/"
img_DIR_P = "../data/images/players/"
img_DIR_T = "../data/images/teams/"
data_DIR = "../data/pbpdata/"
box_DIR = "../data/box/"
aio_DIR = "../data/all_in_one_metrics/"

source = "pbpstats"

In [ ]:
league = "NBA"
season = "2024"
year = season + "-" + str(int(season)+1)[-2:]
with zstd.open(data_DIR + league+"_PBPdata_" + season + ".pkl.zst","rb") as f:
    games_list = dill.load(f)
player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()
len(games_list)

In [ ]:
dfa = []
for game in games_list:
    df1 = pd.DataFrame(game.boxscore.player_items)
    df1["game_id"] = game.game_id
    dfa.append(df1)
df2 = pd.concat(dfa)

In [ ]:
# df2.columns

In [ ]:
cols = ['game_id','team_id', 'team_abbreviation', 'player_id','name', 'status', 'totsec']
df3 = df2[cols]
df3 = df3.rename(columns={"name":"Name"}).reset_index(drop=True)
dfm1 = df3.query("status == 'I'").reset_index(drop=True)

In [ ]:
cols2 = ['PLAYER_ID', 'G', 'Minutes', 'LEBRON', 'LEBRON_WAR', 'multiLEBRON',]
dflb = pd.read_csv(aio_DIR + "NBA_LEBRON_"+ season + ".csv")
dflb = dflb[cols2]
dflb = dflb.rename(columns={"PLAYER_ID":"player_id"})
dflb = dflb.dropna()
dflb["Min"] = dflb["Minutes"]/dflb["G"]
dflb["LBWAR_PG"] = round(dflb["LEBRON_WAR"]/dflb["G"],4)
dflm = dflb[["player_id","LBWAR_PG","Min"]]

In [ ]:
dfm = pd.merge(dfm1,dflm,on="player_id")
dfm = dfm[~dfm.duplicated(keep="last")].reset_index(drop=True)

In [ ]:
len(dfm)-len(dfm1)

In [ ]:
df0 = pd.read_parquet(box_DIR + f"NBA_BOX_T_Base_{season}.parquet")
df0= df0[["GAME_ID","TEAM_ID","GAME_DATE","MATCHUP","WL","PLUS_MINUS"]]
df0 = df0.rename(columns={"GAME_ID":"game_id","TEAM_ID":"team_id"})

In [ ]:
team = "IND"
dfmt =  (
    dfm
    .query(f"team_abbreviation == '{team}'")
    .groupby(["Name"])[["Name","LBWAR_PG","Min"]]
    .agg({"Name":["count"],"Min":["sum"],"LBWAR_PG":["sum"]})
)
dfmt.columns = ["Games","Min","WAR"]
dfmt = dfmt.sort_values("Games",ascending=False).reset_index()
dfmt

In [ ]:
dfmg =  (
    dfm
    .groupby(["game_id","team_id"])[["player_id","LBWAR_PG","Min"]]
    .agg({"player_id":["count"],"Min":["sum"],"LBWAR_PG":["sum"]})
)
dfmg.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
dfmg = dfmg.reset_index()
# merge team boxscores to get game details like date and matchup
dfmf = pd.merge(df0,dfmg,on=["game_id","team_id"])
dfmf["Team"] = dfmf["team_id"].map(teams_dict)
dfmf = dfmf.drop(columns=["team_id","team_id"])
dfmf.insert(1,"Team",dfmf.pop("Team"))
df_x = dfmf.groupby("Team").agg({"Games_Missed":["sum"],"Minutes_Missed":["sum"],"LEBRON_WAR_Missed":["sum"]})
df_x.columns = ["Games_Missed","Minutes_Missed","LEBRON_WAR_Missed"]
df_x = (df_x
        .reset_index()
        .sort_values("LEBRON_WAR_Missed",ascending=False)
        .reset_index(drop=True)
    )
df_x["LEBRON_WAR_Missed"] = df_x["LEBRON_WAR_Missed"].round(2)
df_x["Minutes_Missed"] = df_x["Minutes_Missed"].astype(int)
df_x["image"]= df_x["Team"]
df_x.insert(0,"image",df_x.pop("image"))

In [ ]:
data = df_x.reset_index()
data["index"] +=1
t = (
        GT(data)
        .tab_header(
            title = md("**NBA WAR Missed Leaderboard 2024-25**"),
            subtitle = md("Players listed as Inactive: Missed the game<br />WAR calculated using LEBRON")  
        )
        .tab_source_note(source_note="bsky:@sradjoker.cc | x:@SravanNBA | source: bball-index, pbpstats")
        .cols_label(
            index = "#",
            image = "",
            Games_Missed = "Games Missed",
            Minutes_Missed = "Minutes Missed",
            LEBRON_WAR_Missed="LEBRON WAR"
        )
        .data_color(
            columns=['Games_Missed','Minutes_Missed','LEBRON_WAR_Missed'],
            palette="RdYlGn",
            reverse="True"
        )
        .fmt_image(
            columns="image",
            path = img_DIR_T,
            file_pattern="{}.png"
        )
        .cols_align(align="center")
        .cols_align(align="left", columns="Team")
        .tab_options(
            heading_title_font_size="150%",
            heading_subtitle_font_size="110%",
            # heading_title_font_weight='bold',
            table_background_color="floralwhite",
            column_labels_font_size="105%",
            column_labels_font_weight='bold',
            row_group_font_weight='bold',
            row_group_background_color="#E5E1D8",
            table_font_size=10,
            table_font_names="Consolas", 
            data_row_padding = "0.5px",
            # table_margin_left = 7,
            # table_margin_right = 0,
        )
    )
t.save(fig_DIR + f"NBA_WAR_Missed_{season}.png",scale=3,web_driver="firefox")
t

In [ ]:
df_teams = pd.read_csv("../data/" + "NBA_teams_colors_logos.csv")
df_teams["Team"] = df_teams["nameTeam"]
df_teams = df_teams[["Team","colorsTeam"]]
df_y = pd.merge(df_x, df_teams, on="Team")

In [ ]:
df_avg = df_y.iloc[:,2:-1].mean()
df_avg = pd.DataFrame(df_avg).T
df_avg["Games_Missed"] = df_avg["Games_Missed"].round(0)
df_avg["Minutes_Missed"] = df_avg["Minutes_Missed"].round(1)
df_avg["LEBRON_WAR_Missed"] = df_avg["LEBRON_WAR_Missed"].round(2)
df_avg["Team"] = "League Average"
df_avg["colorsTeam"]  = "#000000"

In [ ]:
df_z = pd.concat([df_y,df_avg]).sort_values("LEBRON_WAR_Missed",ascending=False).reset_index(drop=True)

In [ ]:
teams = df_z["Team"].to_list()
teams.reverse()
df_z["image"]= img_DIR_T + df_z["Team"] + ".png"
df_z["Team"] = pd.Categorical(df_z['Team'], categories=teams)

In [ ]:
from plotnine import theme_classic

In [ ]:
df = df_z
lb_mean = df["LEBRON_WAR_Missed"].mean()
lb_mean = round(lb_mean,2)
# df1 = df.copy()
# df1["LEBRON_WAR_Missed"] = df1["LEBRON_WAR_Missed"] - lb_mean
# df1["LEBRON_WAR_Missed"] = df1["LEBRON_WAR_Missed"].round(3)
p = (
    ggplot(df, aes(x='Team', y='LEBRON_WAR_Missed', image="image"))
    + geom_bar(aes(fill="colorsTeam"),stat="identity", alpha=0.9,show_legend=False)
    + coord_flip()
    + scale_color_identity(aesthetics=["fill"])
    + scale_y_continuous(breaks=np.arange(0,8,1))
    + geom_image(aes(y=-0.2), size=0.05)
    + geom_hline(yintercept=lb_mean, linetype="dashed")
    # + theme_538(base_size=12)
    + theme_classic(base_size=12)
    + theme(
        figure_size=(10,10),
        plot_title=element_text(size=20, weight="bold"),
    )
    + labs(
        title="LEBRON WAR Missed as of "+ datetime.today().strftime("%Y-%m-%d"),
        subtitle=f"Players missing the game due to being Inactive",
        caption="bsky:@sradjoker.cc | x:@SravanNBA | Source: pbpstats, bball-index",
        y= "LEBRON WAR Missed",
        x="",
    )
)
p.save(fig_DIR + f"LEBRON_WAR_Missed_{season}",dpi=300,verbose=False)
p

In [ ]:
lb_mean